# Notebook

Download files 

create spark comands

In [ ]:
!wget -P ../data https://data-engineer-edvai.s3.amazonaws.com/yellow_tripdata_2021-02.parquet
!wget -P ../data https://data-engineer-edvai.s3.amazonaws.com/yellow_tripdata_2021-01.parquet

In [3]:
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext

sc = SparkContext('local')
spark = SparkSession(sc)

from  pyspark.sql import HiveContext
hc = HiveContext(sc)

23/08/27 21:19:24 WARN Utils: Your hostname, rodrigo-Latitude-5400 resolves to a loopback address: 127.0.1.1; using 192.168.0.177 instead (on interface wlo1)
23/08/27 21:19:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/27 21:19:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/27 21:19:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/rodrigo/miniconda3/envs/edvai/lib/python3.10/site-packages/pyspark/sql/context.py:720: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  warnings.warn(
/home/rodrigo/miniconda3/envs/edvai/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSe

In [4]:

# Create a Spark session
spark = SparkSession.builder \
    .appName("ParquetReadExample") \
    .getOrCreate()

# Read Parquet files
parquet_path_1 = "../data/yellow_tripdata_2021-01.parquet"
parquet_path_2 = "../data/yellow_tripdata_2021-02.parquet"

df_01 = spark.read.parquet(parquet_path_1)

df_02 = spark.read.parquet(parquet_path_2)



23/08/27 21:19:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
df_union = df_01.union(df_02)

In [6]:
df_filter = df_union.filter((df_union.airport_fee.isNotNull())&(df_union.airport_fee !=0))

In [7]:
df_filter.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-02-21 05:36:21|  2021-02-21 06:34:17|            3.0|         19.1|       1.0|                 Y|         132|          65|           2|       56.5| 2.25|    0.5|       0.

In [8]:
df_result = df_filter.select(df_filter.tpep_pickup_datetime, df_filter.airport_fee, df_filter.payment_type, df_filter.tolls_amount, df_filter.total_amount)

In [9]:
df_result.show(5)

+--------------------+-----------+------------+------------+------------+
|tpep_pickup_datetime|airport_fee|payment_type|tolls_amount|total_amount|
+--------------------+-----------+------------+------------+------------+
| 2021-02-21 05:36:21|       1.25|           2|         0.0|       59.55|
+--------------------+-----------+------------+------------+------------+



In [ ]:
df_result.write.insertInto('tripdata.airport_trip')